In [1]:
# Реализация гибридной модели рекомендации, которая берёт усреднённый предсказанный рейтинг, рассчитанный алгоритмами Slope One и SVD.
# Импортируем необходимые библиотеки:
# pandas для манипуляции данными,
# scikit-surprise для функциональности рекомендательных систем, такой как чтение датасетов и применение алгоритмов.
import pandas as pd
from surprise import Dataset, Reader, SlopeOne, SVD

# Загружаем датасет с рейтингами фильмов.
# Берём только первые 10 тысяч записей из датасета для ускорения работы. 
ratings = pd.read_csv("ratings.csv").head(10000)

# Создаем таблицу сводки, чтобы увидеть рейтинги, которые пользователи ставят каждому фильму.
# Индекс: userId, Колонки: movieId, Значения: рейтинг.
user_movie_rating = ratings.pivot_table(index='userId', columns='movieId', values='rating')
# Показываем первые 100 строк для быстрого просмотра.
user_movie_rating.head(100)

movieId,1,2,3,4,5,6,7,8,10,11,...,182715,183611,184471,185031,185135,187541,187593,187595,188301,190183
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,NaN,4.0,NaN,NaN,NaN,4.5,NaN,NaN,NaN,NaN,...,4.0,4.0,3.5,4.5,5.0,NaN,4.0,4.0,NaN,NaN
63,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,4.0,NaN,3.5,NaN,NaN,4.5,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
# Определяем объект Reader, указывая шкалу рейтинга.
reader = Reader(rating_scale=(0.5, 5))  # Рейтинги фильмов варьируются от 0.5 до 5.

# Загружаем данные из DataFrame.
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

# Строим полный тренировочный набор данных.
trainset = data.build_full_trainset()

# Инициализируем алгоритмы SlopeOne и SVD
slope_one_alg = SlopeOne()
svd_alg = SVD()

# Обучаем оба алгоритма на полном наборе данных
slope_one_alg.fit(trainset)
svd_alg.fit(trainset)

# Готовим тестовый набор: все комбинации пользователя и предмета, которых нет в наборе обучения
testset = trainset.build_anti_testset()

# Получаем предсказания от обеих моделей
slope_one_predictions = slope_one_alg.test(testset)
svd_predictions = svd_alg.test(testset)
    
# Преобразуем предсказания в датафреймы для более удобной работы с ними
slope_one_df = pd.DataFrame(slope_one_predictions, columns=['userId', 'movieId', 'actual', 'slope_one_est', 'details']).drop(['actual', 'details'], axis=1)
svd_df = pd.DataFrame(svd_predictions, columns=['userId', 'movieId', 'actual', 'svd_est', 'details']).drop(['actual', 'details'], axis=1)

# Объединяем два датафрейма по userId и movieId, чтобы получить оценки Slope One и SVD рядом друг с другом
merged_predictions = pd.merge(slope_one_df, svd_df, on=['userId', 'movieId'])

# Вычисляем среднее значение двух оценок
merged_predictions['mean_rating'] = merged_predictions[['slope_one_est', 'svd_est']].mean(axis=1)

# Преобразуем средние прогнозы в матрицу оценок
mean_predictions_pivot = merged_predictions.pivot_table(index='userId', columns='movieId', values='mean_rating')

# Объединяем исходную таблицу рейтингов со сводной таблицей усредненных прогнозов
# Сначала переиндексируем таблицу mean_predictions_pivot, чтобы убедиться, что она имеет ту же структуру, что и user_movie_rating
mean_predictions_pivot_reindexed = mean_predictions_pivot.reindex_like(user_movie_rating)

# Объединяем фактические рейтинги с предсказанными, заполняя недостающие фактические рейтинги предсказаниями
complete_ratings = user_movie_rating.combine_first(mean_predictions_pivot_reindexed)

# Сортируем столбцы в таблице pivot complete_ratings, чтобы они соответствовали исходному в user_movie_rating
complete_user_movie_rating = complete_ratings.reindex(columns=user_movie_rating.columns)

In [3]:
# Показываем первые 100 строк итоговой таблицы для быстрого просмотра.
complete_user_movie_rating.head(100)

movieId,1,2,3,4,5,6,7,8,10,11,...,182715,183611,184471,185031,185135,187541,187593,187595,188301,190183
userId,,,,,,,,,,,,,,,,,,,,,
1,4.000000,4.334935,4.000000,3.841752,4.041987,4.000000,3.948407,3.513324,4.113717,4.236102,...,4.368307,4.160032,3.771608,4.286023,4.736445,3.949125,4.383400,4.299414,4.256046,4.250078
2,4.033921,3.856128,3.594794,3.098024,3.280335,3.984102,3.322747,2.906368,3.979495,3.665442,...,3.830074,3.721110,3.246723,3.870722,4.287626,3.389962,3.847289,3.786219,4.129001,4.082865
3,2.670215,2.549448,2.313026,2.350305,2.373138,2.422664,2.021953,1.448238,2.473272,2.528570,...,2.574634,2.497572,1.932652,2.514703,2.969289,2.147686,2.346255,2.552262,2.931857,2.845887
4,3.764666,3.343825,3.385426,2.782383,3.411328,3.536855,3.247988,2.851670,3.099667,3.066131,...,3.179909,3.432534,2.820298,3.414248,3.778972,2.892539,3.363019,3.338312,3.356849,3.564987
5,4.000000,3.594662,3.498311,3.212918,3.870365,3.650160,3.286876,3.208505,3.176869,3.384069,...,3.652036,3.623184,3.057160,3.618698,3.957159,3.025114,3.680159,3.672032,3.426915,3.471879
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,4.179050,4.000000,3.888488,3.730881,3.522005,4.500000,3.589599,3.076104,4.319153,3.725303,...,4.000000,4.000000,3.500000,4.500000,5.000000,3.689991,4.000000,4.000000,4.354049,4.414243
63,5.000000,3.458456,3.276935,3.327676,3.198547,3.801988,3.056338,2.604024,3.000000,3.105123,...,3.612521,3.565771,2.909113,3.524057,4.066088,3.235643,3.610298,3.391540,3.695460,3.669690
64,4.000000,3.817392,3.500000,3.329975,3.447323,4.500000,3.298419,2.802589,3.728659,3.604984,...,3.686884,3.676857,3.209229,3.671807,4.085516,3.189302,3.835822,3.666250,3.802971,3.936447


In [4]:
# Экспортируем итоговую таблицу в CSV файл.
complete_user_movie_rating.to_csv("predicted_ratings.csv")